In [1]:
import torch.optim as optim
from utils import *
from model import *
import config
from train import *
from scipy.stats import truncnorm

c:\Users\quock\anaconda3\envs\QuocKhanh\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load weight

In [2]:
gen = Generator(config.Z_DIM, config.IN_CHANNELS, img_channels=config.CHANNELS_IMG).to(config.DEVICE)
critic = Discriminator(config.Z_DIM, config.IN_CHANNELS, img_channels=config.CHANNELS_IMG).to(config.DEVICE)

# initialize optimizers and scalers for FP16 training
opt_gen = optim.Adam(gen.parameters(), lr=config.LEARNING_RATE, betas=(0.0, 0.99))
load_checkpoint('../weightCelebA/generator.pth', gen, opt_gen, config.LEARNING_RATE)

=> Loading checkpoint


In [3]:
def generate_examples(gen, steps, truncation=0.7, n=100):
    """
    Tried using truncation trick here but not sure it actually helped anything, you can
    remove it if you like and just sample from torch.randn
    """
    gen.eval()
    alpha = 1.0
    for i in range(n):
        with torch.no_grad():
            noise = torch.tensor(truncnorm.rvs(-truncation, truncation, size=(1, config.Z_DIM, 1, 1)),
                                 device=config.DEVICE, dtype=torch.float32)
            img = gen(noise, alpha, steps)
            save_image(img * 0.5 + 0.5, f"fakeCeleba/saved_examples/img_{i}.png")
    gen.train()

In [5]:
for i in range(5):
    generate_examples(gen, i, n = 40*(5-i))


In [6]:
import cv2
import matplotlib.pyplot as plt

img = cv2.imread('fakeCeleba/saved_examples/img_187.png', cv2.IMREAD_GRAYSCALE)
img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
# print(img)
plt.imshow(img)

: 

: 

In [ ]:

# Load image in LAB format
lab_img = cv2.imread('saved_examples\img_187.png', cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)

# Convert LAB image to RGB
rgb_img = cv2.cvtColor(lab_img, cv2.COLOR_LAB2RGB)
print(type(rgb_img))
# plt.imshow(rgb_img)

In [5]:
import torch
from pytorch_fid import fid_score

In [7]:
steps = [0,1,2,3,4]
alpha = 1

In [8]:
with torch.no_grad():
    generated_images = []
    noise = torch.tensor(truncnorm.rvs(-0.7, 0.7, size=(1, config.Z_DIM, 1, 1)),
                            device=config.DEVICE, dtype=torch.float32)
    img = gen(noise, alpha, steps[3])
    print(img.shape)
    generated_images.append(img)
    generated_images = torch.cat(generated_images, dim = 0)
    print(generated_images.shape)
loader, datasets = get_loader(4 * 2 ** steps[3])
images, labels = next(iter(loader))

torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])


In [9]:
real = images[0].reshape(1,3,32,32)
real.shape

torch.Size([1, 3, 32, 32])

In [10]:
# print(loader)
# print(generated_images)
fid_value = fid_score.calculate_fid_given_paths([real, generated_images], batch_size=1, device='cuda', dims=0)
# fid_value = fid_score(loader, generated_images)
print("FID: ", fid_value)

TypeError: only integer tensors of a single element can be converted to an index

# from Scratch

In [14]:
inception = torchvision.models.inception_v3(pretrained=True, aux_logits=True)
generate_examples(gen, 3, n = 200)


c:\Users\quock\anaconda3\envs\QuocKhanh\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\quock\anaconda3\envs\QuocKhanh\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [18]:
import torch
from torchvision import datasets, transforms

# Load real images
real_data = datasets.ImageFolder('Celeba_dataset\img_align_celeba', transform=transforms.ToTensor())
real_loader = torch.utils.data.DataLoader(real_data, batch_size=50, shuffle=True)

# Load generated images
fake_data = datasets.ImageFolder('fakeCeleba', transform=transforms.ToTensor())
fake_loader = torch.utils.data.DataLoader(fake_data, batch_size=50, shuffle=True)

# Get a batch of real images and fake images
real_images, _ = next(iter(real_loader))
fake_images, _ = next(iter(fake_loader))


In [21]:
import torchvision.models as models
inception.fc = nn.Identity() # Remove the final fully connected layer
inception.eval() # Set the model to evaluation mode


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [24]:
import numpy as np
from pytorch_fid.inception import InceptionV3
from pytorch_fid.fid_score import calculate_activation_statistics

# Calculate the activation statistics of real images and fake images
mu_real, sigma_real = calculate_activation_statistics(real_images, inception, device='cuda')
mu_fake, sigma_fake = calculate_activation_statistics(fake_images, inception, device='cuda')


  0%|          | 0/1 [00:04<?, ?it/s]


AttributeError: Caught AttributeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "c:\Users\quock\anaconda3\envs\QuocKhanh\lib\site-packages\PIL\Image.py", line 3231, in open
    fp.seek(0)
AttributeError: 'Tensor' object has no attribute 'seek'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\quock\anaconda3\envs\QuocKhanh\lib\site-packages\torch\utils\data\_utils\worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "c:\Users\quock\anaconda3\envs\QuocKhanh\lib\site-packages\torch\utils\data\_utils\fetch.py", line 58, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "c:\Users\quock\anaconda3\envs\QuocKhanh\lib\site-packages\torch\utils\data\_utils\fetch.py", line 58, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "c:\Users\quock\anaconda3\envs\QuocKhanh\lib\site-packages\pytorch_fid

In [ ]:
load_checkpoint('weightCIFAR\generator.pth', gen, opt_gen, config.LEARNING_RATE)
for i in range(5):
    generate_examples(gen, i, n = 40*(i+1))
